In [2]:
import cmdstanpy
cmdstanpy.install_cmdstan()
cmdstanpy.install_cmdstan(compiler=True)

C:\Users\manny\anaconda3\lib\site-packages\pandas\_testing.py:24: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  import pandas._libs.testing as _testing


AttributeError: module 'numpy' has no attribute 'bool'.
`np.bool` was a deprecated alias for the builtin `bool`. To avoid this error in existing code, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

In [9]:
%pip install --upgrade packaging

  Using cached packaging-23.1-py3-none-any.whl (48 kB)
  Attempting uninstall: packaging
    Found existing installation: packaging 20.4
    Uninstalling packaging-20.4:
      Successfully uninstalled packaging-20.4
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install patchworklib

  Using cached dill-0.3.6-py3-none-any.whl (110 kB)
Note: you may need to restart the kernel to use updated packages.


In [4]:
DRAFT = True
import itertools
import logging
import warnings
import numpy as np
print(np.version.version)
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore", "plotnine\..*")

import cmdstanpy as csp
csp.utils.get_logger().setLevel(logging.ERROR)


import statistics as stat
import pandas as pd
import plotnine as pn
import patchworklib as pw

1.23.1
No module named 'seaborn._core'


<Figure size 72x72 with 0 Axes>